# Question Answering over Resources
The notebook is based on https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html

## Prepare Data

In [55]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

import os
# Input OpenAI API Key Here
os.environ["OPENAI_API_KEY"] = "YOUR OPEN_API_KEY"

In [56]:
import json

with open('test_data.json', 'r') as f:
    data = json.load(f)
data = data['data']
num = len(data)
print(num)
print(data[0]['paragraphs'][0]['qas'][0])

35
{'question': 'In what country is Normandy located?', 'id': '56ddde6b9a695914005b9628', 'answers': [{'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}, {'text': 'France', 'answer_start': 159}], 'is_impossible': False}


In [58]:
import openai
model = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
chain = load_qa_with_sources_chain(model, chain_type="stuff")
for i in range(num):
    for j in range(len(data[0]['paragraphs'])):
        text= data[i]['paragraphs'][j]['context']
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        texts = text_splitter.split_text(text)
        embeddings = OpenAIEmbeddings()
        docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])
        for k in range(len(data[i]['paragraphs'][j]['qas'])):
            question = data[i]['paragraphs'][j]['qas'][k]['question']
            if not data[i]['paragraphs'][j]['qas'][k]['is_impossible']:
                answer = data[i]['paragraphs'][j]['qas'][k]['answers'][0]['text']
                pos = data[i]['paragraphs'][j]['qas'][k]['answers'][0]['answer_start']
            else:
                answer = "No information"
                pos = None
            query = question
            docs = docsearch.similarity_search(query, n_results = 1)
            response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
            res_text = response['output_text']
            print(f"question: {question}\nanswer:{answer} \nresponse:{res_text}\n\n\n")
        break
    break

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


question: In what country is Normandy located?,
answer:France, 
response:Normandy is located in France.
SOURCES: 0



question: When were the Normans in Normandy?,
answer:10th and 11th centuries, 
response:The Normans were in Normandy in the 10th and 11th centuries.
SOURCES: 0





Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


question: From which countries did the Norse originate?,
answer:Denmark, Iceland and Norway, 
response:The Norse who originated the Normans came from Denmark, Iceland, and Norway.
SOURCES: 0





Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


...

question: What century did the Normans first gain their separate identity?,
answer:10th century, 
response:The Normans first gained their separate identity in the first half of the 10th century.
SOURCES: 0



question: Who gave their name to Normandy in the 1000's and 1100's,
answer:No information, 
response:The Normans gave their name to Normandy in the 10th and 11th centuries.
SOURCES: 0





Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


question: What is France a region of?,
answer:No information, 
response:Normandy is a region in France.
SOURCES: 0





Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


question: Who did King Charles III swear fealty to?,
answer:No information, 
response:The Normans swore fealty to King Charles III of West Francia.
SOURCES: 0



question: When did the Frankish identity emerge?,
answer:No information, 
response:The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century.
SOURCES: 0





In [ ]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

In [57]:
# Input query here
query = "when is the flight"
docs = docsearch.similarity_search(query, retriever=docsearch.as_retriever(search_kwargs={"k": 1}))

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [35]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

## Quickstart 

In [36]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
chain = load_qa_with_sources_chain(model, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': "I'm sorry, there is no information about a flight in the given content. \nSOURCES:"}